## 1. Library

In [ ]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

## 2. 사이트 정보 추출

In [ ]:
# 빈창 만들기
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')

driver = webdriver.Chrome('chromedriver', options = options)

### 2-1. Selenium 자동 스크롤 설정

In [ ]:
# 사이트 링크 입력
url = "https://www.teamblind.com/kr/search/%EC%A0%84%EA%B8%B0%EC%B0%A8"
driver.get(url)

# 카테고리를 입력해야함
driver.find_element_by_xpath('//*[@id="m_channel"]/option[11]').click()

body = driver.find_element_by_tag_name('body') # body를 스크롤하기 위해 tagname이 body로 되어있는것을 추출.
num_of_pagedowns = 300 # 최대 페이지 내리기

SCROLL_PAUSE_SEC = 1

# 스크롤 높이 가져옴
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 1초 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

print("페이지 읽기 종료")
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.close()

### 2-2. 스크롤 내려오는 동안 url과 포스트명 수집

In [ ]:
anchors = soup.find_all('a', {'class': '', 'href': True})
result = []
text_list = []
for i in anchors :
    result.append(i['href'])
    text_list.append(i.text)

# 추출한 정보들을 데이터프레임으로
link_text = pd.DataFrame({"link" : result, "text" : text_list})
link_text

### 2-3. 추출한 url 필터링

In [ ]:
# 추출한 url이 '/kr/introduce', '/kr/post/%EC%A0%84%EA%B8%B0%EC%B0%A8-%EA%B3%A0' 형식으로 추출됨
# 우리가 필요한건 post에 관한 url임

# 블라인드 홈 url
url = 'https://www.teamblind.com'
url_list = [] # 새롭게 url 저장할 리스트
text_list = []

# url과 게시판 필터링
for i in range(1,len(link_text)) :
    if link_text['text'][i] == "자동차" : # 게시판이 '블라블라', '자동차'일 경우만 가져오기
        url_list.append(url + str(link_text['link'][i+1]))
        text_list.append(str(link_text['text'][i+1]))
        if link_text['link'][i].find('post') == 0 : # url에 'post' text가 없을경우 pass
            pass

# 'https://www.teamblind.com/kr/post/%EC%A0%84%EA', 'https://www.teamblind.com/kr/post/%EC%A0%84%EA' 해당 코드를 통해 url 형식 변경
# '전기차 고민', '전기차 타는 형들'과 같은 포스트 명 추출

In [ ]:
# 가져온 url과 포스트명 데이터프레임으로 저장
url_df = pd.DataFrame({"url": url_list,"text" : text_list})

### 3.  수집한 url을 통해 본문, 댓글 추출

In [ ]:
# 추출한 제목, 내용, 댓글 저장할 리스트
title_list = []
main_list = []
comment_list = []

# Selenium 빈창 만들기
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
driver = webdriver.Chrome('chromedriver', options = options)

# 제목, 내용, 댓글 추출 함수
def blind_scraping(url) :
    url = url
    driver.get(url) # url 가져오기
    time.sleep(0.5)
    titles = driver.find_element_by_xpath('//*[@id="wrap"]/section/div/div[1]/div[1]/h2').text # 해당 url 제목 추출
    main = driver.find_element_by_xpath('//*[@id="contentArea"]').text # 본문 추출
    comment = driver.find_element_by_xpath('//*[@id="wrap"]/section/div/div[1]/div[4]').text # 댓글 추출
    title_list.append(titles)
    main_list.append(main)
    comment_list.append(comment)
    df = pd.DataFrame({'title' : title_list,'main' : main_list,'comment' : comment_list})
    print("정리 끝")
    return df

In [ ]:
# for문으로 url마다 함수 적용
for i in range(0,len(url_df)) :
    url = url_df['url'][i]
    main_text = blind_scraping(url)

In [ ]:
# url 추가해서 데이터프레임 저장 
main_text['url'] = url_df['url']
main_text = main_text[['title','url','main','comment']]

## 4. 파일 저장

In [ ]:
# 크롤링한 파일 xlsx로 저장
main_text.to_excel('블라인드(검색 전기차, 자동차 파트).xlsx',index = None)